## Creating DDL 

In [26]:
import sqlite3

# Crear la conexión a la base de datos
conn = sqlite3.connect('renta_vehiculos.db')

# Crear tabla de vehículos
conn.execute('''CREATE TABLE IF NOT EXISTS vehiculos (
    id INTEGER PRIMARY KEY,
    marca TEXT,
    modelo TEXT,
    anio INTEGER,
    tipo TEXT,
    capacidad INTEGER,
    color TEXT,
    kilometraje REAL,
    estado TEXT
)''')

# Crear tabla de puntos de renta
conn.execute('''CREATE TABLE  IF NOT EXISTS puntos_renta (
    id INTEGER PRIMARY KEY,
    nombre TEXT,
    direccion TEXT,
    telefono TEXT,
    horario_atencion TEXT
)''')

# Crear tabla de clientes
conn.execute('''CREATE TABLE IF NOT EXISTS clientes (
    id INTEGER PRIMARY KEY,
    nombre TEXT,
    correo_electronico TEXT,
    telefono TEXT,
    direccion TEXT
)''')

# Crear tabla de rentas
conn.execute('''CREATE TABLE IF NOT EXISTS rentas (
    id INTEGER PRIMARY KEY,
    id_vehiculo INTEGER,
    id_punto_salida INTEGER,
    id_punto_retorno INTEGER,
    id_cliente INTEGER,
    fecha_inicio TEXT,
    fecha_fin TEXT,
    costo_total REAL,
    duracion_renta TEXT,
    FOREIGN KEY (id_vehiculo) REFERENCES vehiculos(id_vehiculo),
    FOREIGN KEY (id_punto_salida) REFERENCES puntos_renta(id_punto),
    FOREIGN KEY (id_punto_retorno) REFERENCES puntos_renta(id_punto),
    FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente)
)''')

# Extraer datos relevantes de las tablas de vehículos y rentas
query = '''SELECT v.id, v.marca, v.modelo, v.anio, v.tipo, v.capacidad, COUNT(*) AS num_rentas
FROM vehiculos v
INNER JOIN rentas r ON v.id = r.id_vehiculo
WHERE r.fecha_inicio >= date('now', '-1 year')
GROUP BY v.id, v.marca, v.modelo, v.anio, v.tipo, v.capacidad'''

result1 = conn.execute(query).fetchall()

# Calcular el tiempo promedio de renta para cada vehículo
query = '''SELECT r.id_vehiculo, AVG(julianday(r.fecha_fin) - julianday(r.fecha_inicio)) AS duracion_promedio
FROM rentas r
GROUP BY r.id_vehiculo'''

result2 = conn.execute(query).fetchall()

# Agregar los datos a una tabla agregada
query = '''SELECT v.id, v.marca, v.modelo, v.anio, v.tipo, v.capacidad, COUNT(*) AS num_rentas, AVG(julianday(r.fecha_fin) - julianday(r.fecha_inicio)) AS duracion_promedio
FROM vehiculos v
INNER JOIN rentas r ON v.id = r.id_vehiculo
WHERE r.fecha_inicio >= date('now', '-1 year')
GROUP BY v.id, v.marca, v.modelo, v.anio, v.tipo, v.capacidad'''

result3 = conn.execute(query).fetchall()

# Utilizar esta tabla agregada para realizar análisis con diversas dimensiones
query = '''SELECT marca, modelo, anio, tipo, capacidad, num_rentas, duracion_promedio
FROM (
    SELECT v.id, v.marca, v.modelo, v.anio, v.tipo, v.capacidad, COUNT(*) AS num_rentas, AVG(julianday(r.fecha_fin) - julianday(r.fecha_inicio)) AS duracion_promedio
    FROM vehiculos v
    INNER JOIN rentas r ON v.id = r.id_vehiculo
    WHERE r.fecha_inicio >= date('now', '-1 year')
    GROUP BY v.id, v.marca, v.modelo, v.anio, v.tipo, v.capacidad
) AS tabla_agregada
ORDER BY num_rentas DESC, duracion_promedio ASC'''

result4 = conn.execute(query).fetchall()

# Cerrar la conexión a la base de datos
conn.close()

## Creating some fake data 

In [3]:
import pandas as pd
import sqlite3
import random
from faker import Faker

# Crear la conexión a la base de datos
conn = sqlite3.connect('renta_vehiculos.db')

# Crear un objeto Faker para generar datos aleatorios
fake = Faker()

# Generar datos para la tabla de vehículos
vehiculos_data = []
for i in range(1, 101):
    vehiculos_data.append({
        'id_vehiculo': i,
        'marca': fake.company(),
        'modelo':  fake.word(),
        'anio': random.randint(2015, 2023),
        'tipo': random.choice(['Sedan', 'Pickup', 'SUV']),
        'capacidad': random.randint(2, 8),
        'color': fake.color_name(),
        'kilometraje': random.randint(10000, 100000),
        'estado': random.choice(['Disponible', 'En mantenimiento', 'En renta'])
    })

# Generar datos para la tabla de puntos de renta
puntos_renta_data = []
for i in range(1, 11):
    puntos_renta_data.append({
        'id_punto': i,
        'nombre': fake.company(),
        'direccion': fake.address(),
        'telefono': fake.phone_number(),
        'horario_atencion': fake.time(pattern='%H:%M:%S')
    })

# Generar datos para la tabla de clientes
clientes_data = []
for i in range(1, 101):
    clientes_data.append({
        'id_cliente': i,
        'nombre': fake.name(),
        'correo_electronico': fake.email(),
        'telefono': fake.phone_number(),
        'direccion': fake.address()
    })

# Generar datos para la tabla de rentas
rentas_data = []
for i in range(1, 1001):
    rentas_data.append({
        'id_renta': i,
        'id_vehiculo': random.randint(1, 100),
        'id_punto_salida': random.randint(1, 10),
        'id_punto_retorno': random.randint(1, 10),
        'id_cliente': random.randint(1, 100),
        'fecha_inicio': fake.date_time_between(start_date='-1y', end_date='now').strftime('%Y-%m-%d %H:%M:%S'),
        'fecha_fin': fake.date_time_between(start_date='now', end_date='+1y').strftime('%Y-%m-%d %H:%M:%S'),
        'costo_total': random.randint(500, 5000)
    })

# Convertir los datos en DataFrames de pandas
vehiculos_df = pd.DataFrame(vehiculos_data)
puntos_renta_df = pd.DataFrame(puntos_renta_data)
clientes_df = pd.DataFrame(clientes_data)
rentas_df = pd.DataFrame(rentas_data)

# Insertar los datos en las tablas de la base de datos
vehiculos_df.to_sql('vehiculos', conn, if_exists='replace', index=False)
puntos_renta_df.to_sql('puntos_renta', conn, if_exists='replace', index=False)
clientes_df.to_sql('clientes', conn, if_exists='replace', index=False)
rentas_df.to_sql('rentas', conn, if_exists='replace', index=False)

# Cerrar la conexión a la base de datos
conn.close()

In [25]:
import sqlite3

# establecer la ruta de la base de datos
db_file = 'renta_vehiculos.db'

# conectarse a la base de datos
conn = sqlite3.connect(db_file)

# obtener una lista de todas las tablas en la base de datos
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# eliminar cada tabla
for table_name in tables:
    table_name = table_name[0]
    cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
    print(f"La tabla {table_name} ha sido eliminada.")

# confirmar los cambios y cerrar la conexión
conn.commit()
conn.close()

La tabla vehiculos ha sido eliminada.
La tabla puntos_renta ha sido eliminada.
La tabla clientes ha sido eliminada.
La tabla rentas ha sido eliminada.


In [4]:
vehiculos_df 

,id_vehiculo,marca,modelo,anio,tipo,capacidad,color,kilometraje,estado
0,1,"Berry, Jensen and Collins",toward,2017,Sedan,5,PaleTurquoise,70909,Disponible
1,2,Ortega LLC,peace,2019,Sedan,8,RoyalBlue,41372,Disponible
2,3,Fischer-Horne,ago,2017,Sedan,8,BlanchedAlmond,68533,En renta
3,4,Villa-Estrada,scene,2022,Pickup,5,PaleTurquoise,13656,En renta
4,5,"Baldwin, Dougherty and Hawkins",action,2021,Pickup,6,LightCoral,27157,Disponible
...,...,...,...,...,...,...,...,...,...
95,96,Murphy and Sons,anyone,2015,Sedan,7,LightSkyBlue,28464,Disponible
96,97,Chen-Vasquez,Democrat,2019,Pickup,4,Brown,24681,En renta
97,98,Day-Green,candidate,2015,Sedan,6,Teal,15853,En mantenimiento
98,99,"Boyd, Grant and Chapman",student,2022,SUV,5,MediumAquaMarine,61751,Disponible


In [5]:
puntos_renta_df

,id_punto,nombre,direccion,telefono,horario_atencion
0,1,Miller Ltd,"45891 George Estate\nPaulaburgh, AS 78379",(542)436-0842,04:51:52
1,2,Jones Group,"031 Sarah Mill Suite 110\nSharichester, OK 02089",380-602-6088x318,04:59:17
2,3,Hardin Group,"189 James Isle Apt. 453\nLake Aaron, MD 51704",7185650271,02:38:44
3,4,Watson LLC,"12971 Drake Underpass Suite 754\nSarahchester,...",+1-921-462-8706x728,19:48:28
4,5,"Lee, Flores and Moore","694 John Pines\nBarajasberg, RI 35952",907-263-1503,20:26:04
5,6,Tanner-Rice,"6925 Marshall Summit\nPaynemouth, MT 21894",722.540.0325x04271,15:47:42
6,7,Douglas Inc,"6423 Regina Prairie\nPort Mitchellburgh, FM 17525",001-976-474-9809x079,04:08:40
7,8,Barnes LLC,"07676 Jesse Islands\nPort William, FL 48801",956-965-4187,13:48:00
8,9,Walters PLC,USNV Lowe\nFPO AE 91732,001-437-421-3421x0129,05:07:16
9,10,"Grimes, Gordon and Brown","6442 Harper Fords\nNelsonbury, WI 83671",+1-595-299-5838x589,23:15:03


In [6]:
clientes_df

,id_cliente,nombre,correo_electronico,telefono,direccion
0,1,Jordan Gonzalez,jennifer27@example.org,001-751-696-2893x151,"989 Anderson Park\nCastrohaven, NH 55259"
1,2,Marie Rodriguez,erikalarsen@example.org,(204)328-5503x64516,"3410 Miller Village Apt. 934\nVincentfurt, SC ..."
2,3,Tammy Graham,heatheracosta@example.org,001-401-600-8773x4478,"7758 Diaz Freeway\nDanielmouth, KY 58801"
3,4,Joseph Robbins,savannahjohnson@example.com,+1-407-736-2859x9104,"4242 Kim Rue Apt. 079\nWest Andreamouth, UT 94352"
4,5,George Duke,sampsonryan@example.com,001-562-396-1573x3826,"667 Maria Highway\nPort Tylerside, CA 27813"
...,...,...,...,...,...
95,96,Lisa Williams,emilystewart@example.net,(516)679-8094,"710 Sanders Village\nPort Joshua, NM 55683"
96,97,Frank Mejia,guerratiffany@example.net,001-828-707-2608x68813,"2829 Ramos Shoal Suite 931\nLake Kayla, TN 86740"
97,98,David Miranda,williamskatelyn@example.org,+1-218-297-9083x28909,"9404 Jonathon Fords\nHenrychester, MT 53451"
98,99,Steven Hodge,brianhoffman@example.com,(593)819-1588x30107,"141 Obrien Glen Apt. 681\nEast Eric, IN 56582"


In [7]:
rentas_df

,id_renta,id_vehiculo,id_punto_salida,id_punto_retorno,id_cliente,fecha_inicio,fecha_fin,costo_total
0,1,17,10,5,51,2022-08-02 18:47:16,2024-03-17 01:22:33,3966
1,2,92,9,6,92,2023-06-10 18:41:16,2023-09-19 04:41:26,3939
2,3,20,2,6,98,2023-07-07 10:01:24,2023-11-03 14:49:15,4399
3,4,40,9,3,96,2022-08-07 22:43:08,2024-04-18 22:02:36,2939
4,5,12,3,2,85,2023-04-02 20:19:07,2023-09-14 04:45:23,3885
...,...,...,...,...,...,...,...,...
995,996,94,7,10,84,2023-01-27 12:29:05,2024-02-29 19:24:19,869
996,997,77,10,6,32,2022-09-29 11:13:26,2023-09-22 20:48:02,1098
997,998,12,3,6,67,2023-04-05 07:43:44,2024-04-07 19:57:46,571
998,999,82,6,2,71,2022-09-10 00:33:22,2023-11-16 05:12:02,3633


## Generate some csv files  for simulate a etl pipeline

In [8]:
vehiculos_df.to_csv('vehiculos.csv', encoding='utf-8', index=False)

In [9]:
puntos_renta_df.to_csv('puntos_renta.csv', encoding='utf-8', index=False)

In [10]:
clientes_df.to_csv('clientes.csv', encoding='utf-8', index=False)

In [11]:
rentas_df.to_csv('rentas.csv', encoding='utf-8', index=False)

## Generate DDL 

In [18]:
import sqlite3

# Crear una conexión a la base de datos
conn = sqlite3.connect('renta_vehiculos.db')

# Obtener el DDL para cada tabla en la base de datos
cursor = conn.cursor()
cursor.execute("SELECT sql FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

# Mostrar el DDL de cada tabla
for table in tables:
    print(table[0])
    print("\n")

# Cerrar la conexión a la base de datos
conn.close()

CREATE TABLE vehiculos (
    id INTEGER PRIMARY KEY,
    marca TEXT,
    modelo TEXT,
    anio INTEGER,
    tipo TEXT,
    capacidad INTEGER,
    color TEXT,
    kilometraje REAL,
    estado TEXT
)


CREATE TABLE puntos_renta (
    id INTEGER PRIMARY KEY,
    nombre TEXT,
    direccion TEXT,
    telefono TEXT,
    horario_atencion TEXT
)


CREATE TABLE clientes (
    id INTEGER PRIMARY KEY,
    nombre TEXT,
    correo_electronico TEXT,
    telefono TEXT,
    direccion TEXT
)


CREATE TABLE rentas (
    id INTEGER PRIMARY KEY,
    id_vehiculo INTEGER,
    id_punto_salida INTEGER,
    id_punto_retorno INTEGER,
    id_cliente INTEGER,
    fecha_inicio TEXT,
    fecha_fin TEXT,
    costo_total REAL,
    duracion_renta TEXT,
    FOREIGN KEY (id_vehiculo) REFERENCES vehiculos(id_vehiculo),
    FOREIGN KEY (id_punto_salida) REFERENCES puntos_renta(id_punto),
    FOREIGN KEY (id_punto_retorno) REFERENCES puntos_renta(id_punto),
    FOREIGN KEY (id_cliente) REFERENCES clientes(id_cliente)
)




In [22]:
import pandas as pd
import sqlite3
import logging

class RentasETL:

    def __init__(self, vehiculos_file, puntos_renta_file, clientes_file, rentas_file):
        self.vehiculos_file = vehiculos_file
        self.puntos_renta_file = puntos_renta_file
        self.clientes_file = clientes_file
        self.rentas_file = rentas_file
        self.conn = sqlite3.connect('renta_vehiculos.db')
        self.vehiculos_inserted_ids = []
        self.puntos_renta_inserted_ids = []
        self.clientes_inserted_ids = []
        self.rentas_inserted_ids = []
        logging.basicConfig(filename='rentas_etl.log', level=logging.INFO)

    def extract_vehiculos(self):
        vehiculos_df = pd.read_csv(self.vehiculos_file)
        vehiculos_df = vehiculos_df.rename(columns={
            'id_vehiculo': 'id',
            'marca': 'marca',
            'modelo': 'modelo',
            'anio': 'anio',
            'tipo': 'tipo',
            'capacidad': 'capacidad',
            'color': 'color',
            'kilometraje': 'kilometraje',
            'estado': 'estado'
        })
        vehiculos_df.name = 'vehiculos'

        # Verificar si los datos ya existen en la tabla vehiculos antes de cargarlos
        vehiculos_df = vehiculos_df.loc[~vehiculos_df['id'].isin(self.vehiculos_inserted_ids)]
        count = 0  # Inicializar contador
        for index, row in vehiculos_df.iterrows():
            id = row['id']
            cursor = self.conn.cursor()
            cursor.execute(f"SELECT COUNT(*) FROM vehiculos WHERE id = {id}")
            result = cursor.fetchone()
            if result[0] > 0:
                vehiculos_df = vehiculos_df.drop(index)
                logging.info(f"Registro duplicado en la tabla vehiculos: {row}")
            else:
                logging.info(f"Insertando registro en la tabla vehiculos: {row}")
                count += 1  # Incrementar contador en uno

        print(f"Se insertaron {count} nuevos registros en la tabla vehiculos")

        return vehiculos_df

    def extract_puntos_renta(self):
        puntos_renta_df = pd.read_csv(self.puntos_renta_file)
        puntos_renta_df = puntos_renta_df.rename(columns={
            'id_punto': 'id',
            'nombre': 'nombre',
            'direccion': 'direccion',
            'telefono': 'telefono',
            'horario_atencion': 'horario_atencion'
        })


        # Verificar si los datos ya existen en la tabla puntos_renta antes de cargarlos
        puntos_renta_df = puntos_renta_df.loc[~puntos_renta_df['id'].isin(self.puntos_renta_inserted_ids)]
        count = 0  # Inicializar contador
        for index, row in puntos_renta_df.iterrows():
            id = row['id']
            cursor = self.conn.cursor()
            cursor.execute(f"SELECT COUNT(*) FROM  puntos_renta WHERE id = {id}")
            result = cursor.fetchone()
            if result[0] > 0:
                puntos_renta_df = puntos_renta_df.drop(index)
                logging.info(f"Registro duplicado en la tabla puntos_renta: {row}")
            else:
                logging.info(f"Insertando registro en la tabla puntos_renta: {row}")
                count += 1  # Incrementar contador en uno

        print(f"Se insertaron {count} nuevos registros en la tabla puntos_renta")
        return puntos_renta_df

    def extract_clientes(self):
        clientes_df = pd.read_csv(self.clientes_file)
        clientes_df = clientes_df.rename(columns={
            'id_cliente': 'id',
            'nombre': 'nombre',
            'correo_electronico': 'correo_electronico',
            'telefono': 'telefono',
            'direccion': 'direccion'
        })
        clientes_df.name = 'clientes'

        # Verificar si los datos ya existen en la tabla clientes antes de cargarlos
        clientes_df = clientes_df.loc[~clientes_df['id'].isin(self.clientes_inserted_ids)]

        count = 0  # Inicializar contador

        for index, row in clientes_df.iterrows():
            id = row['id']
            cursor = self.conn.cursor()
            cursor.execute(f"SELECT COUNT(*) FROM clientes WHERE id = {id}")
            result = cursor.fetchone()
            if result[0] > 0:
                clientes_df = clientes_df.drop(index)
                logging.info(f"Registro duplicado en la tabla clientes: {row}")
            else:
                logging.info(f"Insertando registro en la tabla clientes: {row}")
                count += 1  # Incrementar contador en uno

        print(f"Se insertaron {count} nuevos registros en la tabla clientes")

        return clientes_df

    def extract_rentas(self):
        rentas_df = pd.read_csv(self.rentas_file)
        rentas_df = rentas_df.rename(columns={
            'id_renta': 'id',
            'id_vehiculo': 'id_vehiculo',
            'id_punto_salida': 'id_punto_salida',
            'id_punto_retorno': 'id_punto_retorno',
            'id_cliente': 'id_cliente',
            'fecha_inicio': 'fecha_inicio',
            'fecha_fin': 'fecha_fin',
            'costo_total': 'costo_total'
        })
        rentas_df.name = 'rentas'

        # Agregar columna de duración de la renta en días
        rentas_df['duracion_renta'] = (pd.to_datetime(rentas_df['fecha_fin']) - pd.to_datetime(rentas_df['fecha_inicio'])).dt.days

        # Eliminar registros con duración negativa o cero
        rentas_df = rentas_df.loc[rentas_df['duracion_renta'] > 0]
   

        # Verificar si los datos ya existen en la tabla rentas antes de cargarlos
        rentas_df = rentas_df.loc[~rentas_df['id'].isin(self.rentas_inserted_ids)]
        count = 0  # Inicializar contador
        for index, row in rentas_df.iterrows():
            id = row['id']
            cursor = self.conn.cursor()
            cursor.execute(f"SELECT COUNT(*) FROM rentas WHERE id = {id}")
            result = cursor.fetchone()
            if result[0] > 0:
                rentas_df = rentas_df.drop(index)
                logging.info(f"Registro duplicado en la tabla rentas: {row}")
            else:
                logging.info(f"Insertando registro en la tabla rentas: {row}")
                count += 1  # Incrementar contador en uno

        print(f"Se insertaron {count} nuevos registros en la tabla rentas")
        return rentas_df

    def transform_vehiculos(self, vehiculos_df):
        vehiculos_df['marca'] = vehiculos_df['marca'].str.capitalize()
        vehiculos_df['modelo'] = vehiculos_df['modelo'].str.capitalize()
        vehiculos_df['tipo'] = vehiculos_df['tipo'].str.capitalize()
        vehiculos_df['color'] = vehiculos_df['color'].str.capitalize()
        return vehiculos_df

    def transform_puntos_renta(self, puntos_renta_df):
        puntos_renta_df['nombre'] = puntos_renta_df['nombre'].str.title()
        puntos_renta_df['direccion'] = puntos_renta_df['direccion'].str.title()
        return puntos_renta_df

    def transform_clientes(self, clientes_df):
        clientes_df['nombre'] = clientes_df['nombre'].str.title()
        clientes_df['correo_electronico'] = clientes_df['correo_electronico'].str.lower()
        clientes_df['direccion'] = clientes_df['direccion'].str.title()
        return clientes_df

    def load_vehiculos(self, vehiculos_df):
        try:
            vehiculos_df.to_sql('vehiculos', self.conn, if_exists='append', index=False)
            self.vehiculos_inserted_ids.extend(vehiculos_df['id'].tolist())
        except Exception as e:
            logging.error(f"Error al cargar la tabla vehiculos: {str(e)}")

    def load_puntos_renta(self, puntos_renta_df):
        try:
            puntos_renta_df.to_sql('puntos_renta', self.conn, if_exists='append', index=False)
            self.puntos_renta_inserted_ids.extend(puntos_renta_df['id'].tolist())
        except Exception as e:
            logging.error(f"Error al cargar la tabla puntos_renta: {str(e)}")

    def load_clientes(self, clientes_df):
        try:
            clientes_df.to_sql('clientes', self.conn, if_exists='append', index=False)
            self.clientes_inserted_ids.extend(clientes_df['id'].tolist())
        except Exception as e:
            logging.error(f"Error al cargar la tabla clientes: {str(e)}")

    def load_rentas(self, rentas_df):
        try:
            rentas_df.to_sql('rentas', self.conn, if_exists='append', index=False)
            self.rentas_inserted_ids.extend(rentas_df['id'].tolist())
        except Exception as e:
            logging.error(f"Error al cargar la tabla rentas: {str(e)}")

    def run(self):
        print("Init the Etl")
        vehiculos_df = self.extract_vehiculos()
        puntos_renta_df = self.extract_puntos_renta()
        clientes_df = self.extract_clientes()
        rentas_df = self.extract_rentas()

        vehiculos_df = self.transform_vehiculos(vehiculos_df)
        puntos_renta_df = self.transform_puntos_renta(puntos_renta_df)
        clientes_df = self.transform_clientes(clientes_df)

        self.load_vehiculos(vehiculos_df)
        self.load_puntos_renta(puntos_renta_df)
        self.load_clientes(clientes_df)
        self.load_rentas(rentas_df)

        self.conn.commit()
        self.conn.close()

In [23]:
etl = RentasETL('vehiculos.csv', 'puntos_renta.csv', 'clientes.csv', 'rentas.csv')

In [24]:
etl.run()

Init the Etl
Se insertaron 0 nuevos registros en la tabla vehiculos
Se insertaron 0 nuevos registros en la tabla puntos_renta
Se insertaron 0 nuevos registros en la tabla clientes
Se insertaron 0 nuevos registros en la tabla rentas
